In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time


In [3]:
def CalculateIntegral(image2D):
    integralImage=[]
    im = image2D
    for i in range(0,im.shape[0]):
        integralImage_row=[]
        for j in range(0,im.shape[1]):
            cornerValue=0
            upperValue=0
            leftValue=0
            value=im[i][j]
            if(i!=0):
                upperValue=integralImage[i-1][j]
            if(j!=0):
                leftValue=integralImage_row[j-1]
            if(i!=0 and j!=0):
                cornerValue=integralImage[i-1][j-1]
            integralImage_row.append(value+upperValue+leftValue-cornerValue)
        integralImage.append(integralImage_row) 
    return np.array(integralImage)  

In [4]:
def CalculateLocalSum(ii2D,point1,point2):
    pA = ii2D[int(point1[0])][int(point1[1])]  
    pB = ii2D[int(point2[0])][int(point1[1])]
    pC = ii2D[int(point1[0])][int(point2[1])]
    pD = ii2D[int(point2[0])][int(point2[1])]
    
    if(int(point1[0])<0):
        pC = 0 
        pA = 0
    if(int(point1[1])<0):
        pB = 0
        pA = 0
    return pD - (pC+ pB) + pA

In [5]:
def EdgeDetect(ii,s):
  first_derivate  = np.zeros((ii.shape[0], ii.shape[1]))
  second_derivate = np.zeros((ii.shape[0], ii.shape[1]))
  if(s % 2 !=0):
    p0 = ((s-1)/2,(s-1)/2)
    p1 = ((ii.shape[0] - s) + ((s-1)/2),   (ii.shape[1] - s) + ((s-1)/2))
  else:
    return -1
  start_time = time.time()
  for i in range(int(p0[0]),int(p1[0])+1):
        for j in range(int(p0[1]),int(p1[1])+1 ):
          p01 = (i - p0[0] - 1, j - p0[1] - 1)
          p11h1 = (i  - 1, j + p0[1])
          p02h1 = (i , j - p0[1] - 1)
          p11h2 = (i  + p0[0], j -1)
          p02h2 = (i- p0[0] - 1 , j )        
          p12 = (i + p0[0], j + p0[1])

          # first derivative
          h1 =  CalculateLocalSum(ii,p01,p11h1) -  CalculateLocalSum(ii,p02h1,p12)
          start_time = time.time()
          h2 = CalculateLocalSum(ii,p02h2,p12) - CalculateLocalSum(ii,p01,p11h2)
          first_derivate[i][j] = int(np.sqrt(h1*h1 + h2*h2))

          # second derivative
          Led = (s**2) * CalculateLocalSum(ii,(i-1, j-1),(i,j))  - CalculateLocalSum(ii,p01,p12)
          second_derivate[i][j]=int(np.abs(Led))  
  
  first_derivate = np.round((first_derivate/np.amax(first_derivate))*255)
  second_derivate = np.round((second_derivate/np.amax(second_derivate))*255)
  return [first_derivate,second_derivate]

In [6]:
def RefineEdge(ii,s,r):
    refine_edge_image= np.zeros((ii.shape[0], ii.shape[1]))
    if(s % 2 !=0):
        p0 = ((s-1)/2,(s-1)/2)
        p1 = ((ii.shape[0] - s) + ((s-1)/2),   (ii.shape[1] - s) + ((s-1)/2))
    else:
        return -1
    for i in range(int(p0[0]),int(p1[0])+1):
            for j in range(int(p0[1]),int(p1[1])+1):
                p01 = (i - p0[0] - 1, j - p0[1] - 1)        
                p12 = (i + p0[0], j + p0[1])
                mean_ij = CalculateLocalSum(ii,p01,p12)/(s**2)
                i_ij = CalculateLocalSum(ii,(i-1,j-1),(i,j))
                if(i_ij > (mean_ij*r)):
                    refine_edge_image[i][j] = i_ij
                else:
                    refine_edge_image[i][j] = 0
    refine_edge_image = np.round((refine_edge_image/np.amax(refine_edge_image))*255)
    return refine_edge_image


In [1]:
def convertImage(path):
    return np.asarray(Image.open(path).convert('L'))
